In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import plotly.express as px

from clustering import PhaseFieldModelPreProcessor, SliceSelection

In [2]:
wk_path = '/path/to/matflow/workflow'
pre_proc = PhaseFieldModelPreProcessor(workflow_dir=wk_path)

Loading matflow config from C:\Users\adamj\.matflow\config.yml
Loading task schemas from 1 file(s)...OK!
Loading software definitions from 1 file(s)...OK!


In [3]:
# Slice selection and clustering parametrisation:
slice_selection = SliceSelection(
    increment_idx=-1,
    is_periodic=True,
    x=None,
    y=0,
    z=None,
    eye='+y',
    up='+z',
)

In [4]:
clust_params = {
    'element_idx': 0,
    'slice_selection': slice_selection,
    'method': 'MTEX-FMC', # fast-multiscale clustering, using MTEX
    'parameters': {
        'C_Maha': 2.0,
        'smoothing': 16,    
        'MTEX_script_path': 'segment_slice_new.m',
        'specimen_symmetry': 'triclinic',
        'crystal_symmetries': {
            'Ti-alpha': { # keyed by DAMASK phase name
                'symmetry': 'hexagonal',
                'mineral': 'Ti-alpha',
                'unit_cell_alignment': {'x': 'a'}, 
            },
            'Ti-beta': {
                'symmetry': 'cubic',
                'mineral': 'Ti-beta',
                'unit_cell_alignment': {'x': 'a'},
            }
        },
    },
}

clst = pre_proc.get_clusterer(**clust_params)

In [5]:
# clst.load()
clst.do_segmentation(redo=True) # also does post-processing and saves results

2022-04-13 15:39:57,330 INFO clustering: Running segmentation...
2022-04-13 15:39:57,795 INFO clustering: Running MTEX script...
2022-04-13 15:44:18,141 INFO clustering: Finished MTEX script.
2022-04-13 15:44:18,142 INFO clustering: Starting post-processing...
2022-04-13 15:44:18,143 INFO clustering: Accounting for periodicity...
2022-04-13 15:44:18,144 INFO clustering: Saving 'MTEX_FMC_Clusterer' to file C:\Users\adamj\Dropbox (The University of Manchester)\sims_db\matflow\dual-phase-ti\dual_phase_ti_64_grid_2021-08-17-165641\segmentation_new\element_0__inc_-1__slice_y_0__periodic__method_MTEX-FMC__eye_+y__up_+z__C_Maha_2.00__smoothing_16\clusterer.json...
2022-04-13 15:44:20,347 INFO clustering: Done.
2022-04-13 15:44:20,348 INFO clustering: Finished.


In [8]:
px.imshow(clst.grain_IDs)

In [9]:
clst.show_estimated_dislocation_density()

FigureWidget({
    'data': [{'coloraxis': 'coloraxis',
              'type': 'heatmap',
              'uid': '…

In [10]:
clst.set_seed_points(
    method='dislocation_density',
    pixel_length=0.4e-6,
    # random_seed=413402,
) 
clst.tessellate_seed_points()
clst.save() # re-save with the additional tessellation info

number of seed points set: 28
num_cipher_phases: 32


In [18]:
clst.show_field_data('phase')

FigureWidget({
    'data': [{'colorbar': {'tickmode': 'array',
                           'ticktext': [Ti-alph…

In [19]:
clst.show_tessellated_sub_grain_IDs()

FigureWidget({
    'data': [{'coloraxis': 'coloraxis',
              'type': 'heatmap',
              'uid': '…

## Generate CIPHER input file

In [5]:
clst.load()

2022-04-13 16:24:49,435 INFO clustering: Loading 'MTEX_FMC_Clusterer' from file C:\Users\adamj\Dropbox (The University of Manchester)\sims_db\matflow\dual-phase-ti\dual_phase_ti_64_grid_2021-08-17-165641\segmentation_new\element_0__inc_-1__slice_y_0__periodic__method_MTEX-FMC__eye_+y__up_+z__C_Maha_2.00__smoothing_16\clusterer.pkl...
2022-04-13 16:24:49,582 INFO clustering: Done.


In [12]:
clst.write_CIPHER_input_data(
    path='CIPHER_input_test.yaml',
    materials={
        'Ti-alpha': {
            "chemicalenergy": "none",
            "molarvolume": 1e-5,
            "temperature0": 500.0,            
        },
        "Ti-beta": {
            "chemicalenergy": "none",
            "molarvolume": 1e-5,
            "temperature0": 500.0,
        },        
    },
    interfaces={
        "Ti-alpha-Ti-alpha-HAGB": {
            "energy": {"e0": 0.5},
            "mobility": {"m0": 1.0e-11},
        },
        "Ti-alpha-Ti-beta": {
            "energy": {"e0": 0.5},
            "mobility": {"m0": 1.0e-11},
        },
        "Ti-beta-Ti-beta-HAGB": {
            "energy": {"e0": 0.5},
            "mobility": {"m0": 1.0e-11},
        },
        "Ti-alpha-Ti-alpha-LAGB": {
            "energy": {"e0": 0.2},
            "mobility": {"m0": 5.0e-11},
        },
        "Ti-beta-Ti-beta-LAGB": {
            "energy": {"e0": 0.2},
            "mobility": {"m0": 5.0e-11},
        },
    },
    solution_parameters={
        "time": 1000000,
        "initblocksize": [1, 1],
        "initrefine": 8,
        "minnrefine": 0,
        "initcoarsen": 6,
        "amrinterval": 25,
        "outputfreq": 100,
        "outfile": "test_run",
        "interpolation": "cubic",
        "reltol": 1e-4,
        "abstol": 1e-4,
        "petscoptions": "-ts_adapt_monitor -ts_rk_type 2a",
        "random_seed": 1579993586,
    }
)